In [1]:
import json
import numpy as np
import pymysql
import jieba
import re
import keras

Using TensorFlow backend.


In [2]:
conn = pymysql.connect('localhost','root','9432','companyb',charset='utf8')
cursor=conn.cursor()
#选取所有投资关系
cursor.execute("select investor,company from pre_rel where location is not null and locationi is not null")
rel=cursor.fetchall()

In [1]:
import numpy as np

In [4]:
com.shape

(619972, 325)

In [3]:
#企业对应编号
comnum=json.load(open('data/comnum.json','r'))
#企业信息矩阵
com=np.load('data/com.npy')
com[:,124]=0

In [4]:
#投资关系对应的企业代码
rela=[]
for i,r in enumerate(rel):
    rela.append([comnum[r[0]],comnum[r[1]]])

In [7]:
#投资关系转换成dict
dic={}
for i in range(com.shape[0]):
    dic[i]=[]
for r in rela:
    dic[r[0]].append(r[1])

In [6]:
model=keras.models.load_model('model/keras/keras')

In [16]:
le=np.ones((10000,com.shape[1]),dtype=np.float32)*com[5]

In [17]:
ri=com[:10000]

In [9]:
dic0

{0: [],
 1: [108566],
 2: [],
 3: [],
 4: [],
 5: [44110],
 6: [462279, 503934],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [295579],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [],
 18: [],
 19: [],
 20: [],
 21: [],
 22: [122765, 133091, 104049, 103306],
 23: [],
 24: [],
 25: [],
 26: [],
 27: [],
 28: [],
 29: [],
 30: [],
 31: [],
 32: [],
 33: [104045],
 34: [],
 35: [],
 36: [],
 37: [],
 38: [],
 39: [],
 40: [],
 41: [],
 42: [],
 43: [],
 44: [],
 45: [],
 46: [],
 47: [],
 48: [],
 49: [126173],
 50: [],
 51: [],
 52: [],
 53: [],
 54: [],
 55: [],
 56: [],
 57: [],
 58: [],
 59: [],
 60: [],
 61: [],
 62: [],
 63: [],
 64: [505542],
 65: [88367, 121885],
 66: [151382],
 67: [425670],
 68: [104619, 67478],
 69: [],
 70: [],
 71: [],
 72: [],
 73: [187567, 219963],
 74: [519297, 198758, 327931],
 75: [361942, 465335],
 76: [],
 77: [],
 78: [369330, 140260],
 79: [],
 80: [127328],
 81: [84880, 313235, 315390, 314388],
 82: [],
 83: [],
 84: [],
 85: [],
 86: [],
 87: [],
 8

In [18]:
al=np.hstack((le,ri))

In [19]:
ts=model.predict(al)

In [20]:
tr=model.predict(al).argmax(axis=1)

In [21]:
ts[:,0]=[i for i in range(ts.shape[0])]

In [22]:
tsl=ts.tolist()
tsl.sort(key=lambda x:x[1],reverse=True)

In [25]:
np.array(tsl[:100])[:,0].astype(np.int).tolist()

[1344,
 62,
 135,
 575,
 40,
 1703,
 319,
 23,
 56,
 30,
 389,
 1507,
 607,
 287,
 18,
 502,
 1750,
 1351,
 684,
 36,
 1279,
 1408,
 17,
 430,
 280,
 2122,
 55,
 27,
 2103,
 20,
 282,
 19,
 667,
 1275,
 44,
 60,
 1356,
 53,
 1820,
 54,
 5235,
 51,
 128,
 49,
 7348,
 1778,
 1272,
 1723,
 21,
 225,
 16,
 124,
 342,
 2123,
 1227,
 697,
 1088,
 67,
 975,
 50,
 74,
 1370,
 2983,
 1783,
 33,
 1135,
 428,
 7850,
 723,
 129,
 475,
 6604,
 69,
 589,
 490,
 491,
 2362,
 7670,
 5791,
 8217,
 993,
 8720,
 1668,
 359,
 1826,
 2292,
 7970,
 213,
 1844,
 8172,
 432,
 163,
 484,
 121,
 4202,
 1199,
 1367,
 73,
 332,
 209]

In [9]:
np.ceil(9.2)

10.0

In [32]:
c10=0
c100=0
c1000=0
inum=0#统计已出现的投资者
rnum=0#统计已出现的投资关系
for i,k in enumerate(dic.keys()):
    if len(dic[k])>0:
        inum+=1;rnum+=len(dic[k])
        tsl=[]
        for j in range(com.shape[0]//100000+1):
            ri=com[j*100000:(j+1)*100000]
            le=np.ones((len(ri),com.shape[1]),dtype=np.float32)*com[k]
            al=np.hstack((le,ri))
            ts=model.predict(al)
            ts[:,0]=[m for m in range(j*100000,j*100000+len(ri))]
            tsl+=ts.tolist()
        tsl.sort(key=lambda x:x[1],reverse=True)
        t1000=np.array(tsl[:1000])[:,0].astype(np.int).tolist()
        t100=t1000[:100]
        t10=t100[:10]
        for v in dic[k]:
            if v in t10:
                c10+=1;c100+=1;c1000+=1
            elif v in t100:
                c100+=1;c1000+=1
            elif v in t1000:
                c1000+=1
        print(i,'top10:',c10/inum/10,c10/rnum,'top100:',c100/inum/100,c100/rnum,'top1000:',c1000/inum/1000,c1000/rnum)

 553 top10: 0.0018867924528301887 0.008869179600886918 top100: 0.0008490566037735849 0.03991130820399113 top1000: 0.0003584905660377358 0.168514412416851446510: 0.0 0.0 top100: 0.0010526315789473684 0.05263157894736842 top1000: 0.00021052631578947367 0.10526315789473684top10: 0.0 0.0 top100: 0.0006060606060606061 0.030303030303030304 top1000: 0.0003939393939393939 0.196969696969696960.0005555555555555556 0.02857142857142857 top1000: 0.0003888888888888889 0.20.0 top100: 0.0007142857142857143 0.03571428571428571 top1000: 0.00035714285714285714 0.17857142857142858 top10: 0.0 0.0 top100: 0.0006976744186046512 0.03488372093023256 top1000: 0.0003488372093023256 0.1744186046511628 top100: 0.0008888888888888889 0.0449438202247191 top1000: 0.00035555555555555557 0.1797752808988764 top1000: 0.00032 0.1702127659574468172 top10: 0.00196078431372549 0.010526315789473684 top100: 0.000784313725490196 0.042105263157894736 top1000: 0.00031372549019607844 0.16842105263157894 0.0018867924528301887 0.0103

KeyboardInterrupt: 

In [31]:
com[:100].shape

(100, 325)

In [ ]:
co

In [104]:
comnum2=dict(zip(comnum.values(),comnum.keys()))

In [18]:
inv=[]
for d in dic0.keys():
    if len(dic0[d])>0:
        inv.append(d)
print(len(inv))

245736


In [34]:
count=0
for r in rela:
    print('\r'+str(r[0]),end='')
    rn=np.hstack((com[r[0]],com[r[1]])).reshape(1,650)
    count+=model.predict(rn).argmax()
print(count/len(rela))

6198870.864896384677


In [37]:
count/len(rela)

0.86489638467749175

In [39]:
rn=np.hstack((com[5],com[5])).reshape(1,650)
model.predict(rn)

array([[ 0.22839861,  0.77160138]], dtype=float32)

In [31]:
com[44110]

array([ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [40]:
a1=np.load('data/reltrue.npy')
a1=a1.astype(np.float32)
np.save('data/reltrue.npy',a1)

In [2]:
a1=np.load('data/relfalse.npy')
a1=a1.astype(np.float32)
np.save('data/relfalse.npy',a1)